In [0]:
pip install --upgrade google-cloud-vision

In [0]:
import io
import os

# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types

from PIL import Image, ImageDraw

# Instantiates a client

#uncomment on 1st use
#credientials_path = "/content/hackathon-covid-4a34b61cd5a4.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credientials_path
client = vision.ImageAnnotatorClient()

# The name of the image file to annotate
img_file = os.path.abspath('/content/image_test.png')

def detect_face(face_file, max_results=6):
        """Uses the Vision API to detect faces in the given file.

        Args:
            face_file: A file-like object containing an image with faces.

        Returns:
            An array of Face objects with information about the picture.
        """
        client = vision.ImageAnnotatorClient()

        content = face_file.read()
        image = types.Image(content=content)

        return client.face_detection(
            image=image, max_results=max_results).face_annotations

def highlight_faces(image, faces, output_filename):
        """Draws a polygon around the faces, then saves to output_filename.

        Args:
          image: a file containing the image with the faces.
          faces: a list of faces found in the file. This should be in the format
              returned by the Vision API.
          output_filename: the name of the image file to be created, where the
              faces have polygons drawn around them.
        """
        im = Image.open(image)
        draw = ImageDraw.Draw(im)
        # Sepecify the font-family and the font-size
        for face in faces:
            box = [(vertex.x, vertex.y)
                   for vertex in face.bounding_poly.vertices]
            draw.line(box + [box[0]], width=5, fill='#00ff00')
            # Place the confidence value/score of the detected faces above the
            # detection box in the output image
            draw.text(((face.bounding_poly.vertices)[0].x,
                       (face.bounding_poly.vertices)[0].y - 30),
                      str(format(face.detection_confidence, '.3f')) + '%',
                      fill='#FF0000')
        im.save(output_filename)

with open(img_file, 'rb') as image:
  faces = detect_face(image)
  print('Found {} face{}'.format(
      len(faces), '' if len(faces) == 1 else 's'))

  # Reset the file pointer, so we can read the file again
  image.seek(0)
  highlight_faces(image, faces, '/content/img_w_highlights.png')

In [0]:
import io, os
from numpy import random
from google.cloud import vision
import pandas as pd

credientials_path = "/content/hackathon-covid-4a34b61cd5a4.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credientials_path
client = vision.ImageAnnotatorClient()

file_name = 'is_200327_doctor_mask_800x450.jpg'
image_path = os.path.join('/content', file_name)


with io.open(image_path, 'rb') as image_file:
    content = image_file.read()

image = vision.types.Image(content=content)
response = client.object_localization(image=image)
localized_object_annotations = response.localized_object_annotations

pillow_image = Image.open(image_path)
df = pd.DataFrame(columns=['name', 'score'])
for obj in localized_object_annotations:
    df = df.append(
        dict(
            name=obj.name,
            score=obj.score
        ),
        ignore_index=True)
    
    r, g, b = random.randint(150, 255), random.randint(
        150, 255), random.randint(150, 255)


print(df)
pillow_image.show()

In [0]:
def localize_objects(path):
        """Localize objects in the local image.

        Args:
        path: The path to the local file.
        """
        from google.cloud import vision
        client = vision.ImageAnnotatorClient()

        with open(path, 'rb') as image_file:
            content = image_file.read()
        image = vision.types.Image(content=content)

        objects = client.object_localization(
            image=image).localized_object_annotations

        print('Number of objects found: {}'.format(len(objects)))
        for object_ in objects:
            print('\n{} (confidence: {})'.format(object_.name, object_.score))
            print('Normalized bounding polygon vertices: ')
            for vertex in object_.bounding_poly.normalized_vertices:
                print(' - ({}, {})'.format(vertex.x, vertex.y))

def highlight_faces(image, objects, output_filename):
        """Draws a polygon around the faces, then saves to output_filename.

        Args:
          image: a file containing the image with the faces.
          faces: a list of faces found in the file. This should be in the format
              returned by the Vision API.
          output_filename: the name of the image file to be created, where the
              faces have polygons drawn around them.
        """
        im = Image.open(image)
        draw = ImageDraw.Draw(im)
        # Sepecify the font-family and the font-size
        for objet in objects:
            box = [(vertex.x, vertex.y)
                   for vertex in objet.bounding_poly.vertices]
            draw.line(box + [box[0]], width=5, fill='#00ff00')
            # Place the confidence value/score of the detected faces above the
            # detection box in the output image
            draw.text(((objet.bounding_poly.vertices)[0].x,
                       (objet.bounding_poly.vertices)[0].y - 30),
                      str(format(face.detection_confidence, '.3f')) + '%',
                      fill='#FF0000')
        im.save(output_filename)

with open(img_file, 'rb') as image:
  objects = client.object_localization(
            image=image).localized_object_annotations
  print('Found {} face{}'.format(
      len(objects), '' if len(objects) == 1 else 's'))

  # Reset the file pointer, so we can read the file again
  image.seek(0)
  highlight_faces(image, objects, '/content/img_w_highlights2.png')
